https://www.kaggle.com/competitions/spaceship-titanic/data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import csv

#讀取資料
train_df="/content/drive/MyDrive/AI/train_test_csv/spaceship-titanic/train.csv"
test_df="/content/drive/MyDrive/AI/train_test_csv/spaceship-titanic/test.csv"
train_df = pd.read_csv(train_df)
test_df = pd.read_csv(test_df)

In [ ]:
train_df.head(10)

In [ ]:
train_df.info()

In [ ]:
#因為資料太多且雜 做一個表格列出空格多的特真
import matplotlib.pyplot as plt
import seaborn as sns
#創建一個寬度為 24 單位、高度為 8 單位的新圖形
plt.figure(figsize=(24,8))
#train_df.isnull(): 創train_df的副本(cols_with_null)，相應train_df 把有資料設True或沒資料設False。
#.sum(): 將每個欄位上的 True 值相加, 看哪個特徵缺最多資料。
#.sort_values(ascending=False): 降冪排列
cols_with_null=train_df.isnull().sum().sort_values(ascending=False)
# 設置表格x y 軸
sns.barplot(x=cols_with_null.index,y=cols_with_null)
# 旋轉表格角度
plt.xticks(rotation=90)
plt.show;

In [ ]:
import pandas as pd
# 使用 str.split 函數將一列資料拆分成三列
train_df[['deck', 'num', 'side']] = train_df["Cabin"].str.split('/', expand=True)
test_df[['deck', 'num', 'side']] = test_df["Cabin"].str.split('/', expand=True)
# 刪除原始的 col1 列
train_df = train_df.drop('Cabin', axis=1)
test_df = test_df.drop('Cabin', axis=1)

print(train_df.head(10))


In [ ]:
#把資料變得可計算(自動改)
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
train_df[['PassengerId']] = ordinal_encoder.fit_transform(train_df[['PassengerId']])
train_df[['HomePlanet']] = ordinal_encoder.fit_transform(train_df[['HomePlanet']])
train_df[['CryoSleep']] = ordinal_encoder.fit_transform(train_df[['CryoSleep']])
train_df[['Destination']] = ordinal_encoder.fit_transform(train_df[['Destination']])
train_df[['VIP']] = ordinal_encoder.fit_transform(train_df[['VIP']])
train_df[['Transported']] = ordinal_encoder.fit_transform(train_df[['Transported']])
train_df[['deck']] = ordinal_encoder.fit_transform(train_df[['deck']])
train_df[['num']] = ordinal_encoder.fit_transform(train_df[['num']])
train_df[['side']] = ordinal_encoder.fit_transform(train_df[['side']])
train_df = train_df.drop(['Name','PassengerId'], axis=1)
#'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck','Destination','VIP','side'
test_df[['PassengerId']] = ordinal_encoder.fit_transform(test_df[['PassengerId']])
test_df[['HomePlanet']] = ordinal_encoder.fit_transform(test_df[['HomePlanet']])
test_df[['CryoSleep']] = ordinal_encoder.fit_transform(test_df[['CryoSleep']])
test_df[['Destination']] = ordinal_encoder.fit_transform(test_df[['Destination']])
test_df[['VIP']] = ordinal_encoder.fit_transform(test_df[['VIP']])
test_df[['deck']] = ordinal_encoder.fit_transform(test_df[['deck']])
test_df[['num']] = ordinal_encoder.fit_transform(test_df[['num']])
test_df[['side']] = ordinal_encoder.fit_transform(test_df[['side']])
test_df = test_df.drop(['Name','PassengerId'], axis=1)

#填補空缺資料
train_df = train_df.fillna(train_df.mean(numeric_only=True))
test_df = test_df.fillna(test_df.mean(numeric_only=True))

#訓練的(題目:x 答案:y)
x_train=train_df.drop(['Transported'],axis=1)
y_train=train_df['Transported']

In [ ]:
mean=x_train.mean(axis=0)
x_train-=mean
std=x_train.std(axis=0)
x_train/=std
test_df-=mean
test_df/=std

In [ ]:
from sklearn.model_selection import train_test_split
xx_train, x_val, yy_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential([
    layers.Dense(256,activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(64,activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(64,activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(32,activation="relu"),
    layers.BatchNormalization(),
    layers.Dense(32,activation="sigmoid"),
    layers.BatchNormalization(),
    layers.Dense(16,activation="sigmoid"),
    layers.Dense(4,activation="sigmoid"),
    layers.Dense(1,activation="sigmoid")
])


In [17]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',  # 使用二元交叉熵
              metrics=['accuracy']
             )

In [ ]:
history=model.fit(
    xx_train,
    yy_train,
    epochs=100,
    batch_size=32,
    validation_data=(x_val,y_val)

)

Epoch 1/100
218/218 [==============================] - 5s 6ms/step - loss: 0.6566 - accuracy: 0.5510 - val_loss: 0.6049 - val_accuracy: 0.7602
Epoch 2/100
218/218 [==============================] - 1s 4ms/step - loss: 0.5709 - accuracy: 0.7872 - val_loss: 0.5698 - val_accuracy: 0.7717
Epoch 3/100
218/218 [==============================] - 1s 5ms/step - loss: 0.5416 - accuracy: 0.7877 - val_loss: 0.5435 - val_accuracy: 0.7729
Epoch 4/100
218/218 [==============================] - 1s 5ms/step - loss: 0.5181 - accuracy: 0.7962 - val_loss: 0.5236 - val_accuracy: 0.7809
Epoch 5/100
218/218 [==============================] - 1s 4ms/step - loss: 0.5072 - accuracy: 0.7934 - val_loss: 0.5230 - val_accuracy: 0.7855
Epoch 6/100
218/218 [==============================] - 1s 5ms/step - loss: 0.4970 - accuracy: 0.7968 - val_loss: 0.5070 - val_accuracy: 0.7809
Epoch 7/100
218/218 [==============================] - 1s 5ms/step - loss: 0.4829 - accuracy: 0.8031 - val_loss: 0.5015 - val_accuracy: 0.7895

In [ ]:
from sklearn.feature_selection import mutual_info_regression
import matplotlib.pyplot as plt

X = xx_train
y = yy_train
discrete_features='auto'

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 將 NumPy 數組轉換為 Pandas Series
mi_scores_series = pd.Series(mi_scores, name="MI Scores")

# 繪製長條圖
mi_scores_series.plot(kind='bar', figsize=(10, 6))
plt.title('feature_scores')
plt.xlabel('feature')
plt.ylabel('mi_scores')
plt.show()
